<a href="https://www.inove.com.ar"><img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/PA%20Banner.png" width="1000" align="center"></a>


# kmeans vs GaussianMixture

Ejemplo: Comparación entre
kmeans y GaussianMixture
<br>
v1.1

### Objetivos: 
*  Construir modelos kmeans y GaussianMixture empleando el dataset "FuelConsumptionCo2".
*  Comparar los resultados de los modelos kmeans y GaussianMixture en el dataset "FuelConsumptionCo2".

In [ ]:
#Librerias a implementar
import os
import platform

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Recolectar datos
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline1.png" width="1000" align="middle">

### Código de descarga del dataset

In [ ]:
if os.access('FuelConsumptionCo2.csv', os.F_OK) is False:
    if platform.system() == 'Windows':
        !curl https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/FuelConsumptionCo2.csv > FuelConsumptionCo2.csv
    else:
        !wget FuelConsumptionCo2.csv https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/FuelConsumptionCo2.csv

# Procesar datos
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline2.png" width="1000" align="middle">

In [ ]:
# Realizar una inspeccion del dataset en búsqueda de elementos faltantes
# Una vez descargado el archivo en Colab.
# Leerlo con Pandas y el método read_csv
# Una vez extraida toda la información se almacena en df
# A partir de df y el método describe(), mostrará la descripción estadistica básica del archivo que se guardará en des
# Crear una fila nueva llamada Nan en el DataFrame  des,
# que indica la cantidad de datos tipo Nan que tiene cada columna.
# Para crear una nueva fila, se utilizará el operador loc, donde se indica el nombre
# de la nueva fila y con que valores se completará.
# La información será de los datos faltantes df.isna().sum()
# Crear una fila nueva llamada %Nan en el DataFrame des,
# Esta fila se completará con los porcentajes de Nan encontrados en cada columna.
df = pd.read_csv("FuelConsumptionCo2.csv")
des = df.describe()
des.loc['Nan'] = df.isna().sum()
des.loc['%Nan'] = (df.isna().mean())*100
des

In [ ]:
# Muestra las 5 primeras filas
df.head()

In [ ]:
# Se hace una copia
df1 = df.copy()

# Cambiando los nombres de las columnas
df1.columns = ['AÑO', 'MARCA', 'MODELO', 'CLASE_VEHICULO', 'TAMAÑO_MOTOR', 'CILINDROS', 'TRANSMISION', 'TIPO_COMBUSTIBLE', 'CONSUMO_COMBUSTIBLE_CIUDAD', 'CONSUMO_COMBUSTIBLE_HWY', 'CONSUMO_COMBUSTIBLE_COMB', 'CONSUMO_COMBUSTIBLE_COMB_MPG', 'EMISIONES_CO2']

# Muestra las 5 primeras filas
df1.head()

## Fin de la limpieza
Se finalizó la limpieza, no hay datos mal cargados o incompletos en este dataset

In [ ]:
# Cantidad de filas y columnas con shape
# En la ubicación 0 corresponde a las filas
print('Cantidad de datos en observacion:', df.shape[0])

# Explorar datos
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline3.png" width="1000" align="middle">

In [ ]:
# Analizaremos como se compartan algunos atributos de entrada contra el objetivo (las emisiones, el Co2)
# sns, alias de Seaborn
# data, indicar todos los datos del DataFrame
# x_vars, indicar las columnas a representar en el eje x
# y_vars, indicar la columna a relacionar
# height=5, altura de los gráficos
# diag_kind=None, para indicar el tipo de diagonal, al ser None, significa ninguna.
pp = sns.pairplot(data=df1, x_vars=['CILINDROS', 'TAMAÑO_MOTOR', 'CONSUMO_COMBUSTIBLE_COMB'], y_vars=['EMISIONES_CO2'], diag_kind=None, height=5)
plt.show()


Analizaremos la relación entre le consumo de combustible (FUELCONSUMPTION_COMB) y la emisión de CO2

Se puede observar que está centrada a la izquierda la distribución y no hay outliers, por lo que no habrá problema. El hecho de que la distribución esté marcada a la izquierda querra decir que hay menos datos para analizar de alta cilindrada.

In [ ]:
# Representación gráfica del consumo de combustible y las emisiones CO2
# a través de un gráfico de dispersión.
fig = plt.figure()
ax = fig.add_subplot()
ax = sns.scatterplot(x=df1['CONSUMO_COMBUSTIBLE_COMB'], y=df1['EMISIONES_CO2'], color='darkCyan', ax=ax)
ax.grid('dashed')
plt.show()

# Entrenar modelo (solo regresión lineal)
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline4.png" width="1000" align="middle">

#### **Crear un modelo con Kmeans**
Objetivo: Separar las rectas en 3 conjuntos (clusters) distintos utilizando algún algoritmo de segmentación Kmeans.

In [ ]:
# Muestra las 5 primeras filas.
df1.head()

In [ ]:
# Eliminar las columnas que contienen datos tipo strings, como: 'AÑO', 'MARCA', 'MODELO', 'CLASE_VEHICULO','TRANSMISION', 'TIPO_COMBUSTIBLE'
df2 = df1.drop(['AÑO', 'MARCA', 'MODELO', 'CLASE_VEHICULO','TRANSMISION', 'TIPO_COMBUSTIBLE'], axis=1)
df2.head()

In [ ]:
# Recordar que el Kmeans es un algoritmo de agrupamiento, donde no se sabe a que grupo perteneces los datos.
# Por lo tanto, el algoritmo requiere toda la información numérica.
X_kmeans = df2.values

In [ ]:
# Entrenar diferentes modelos de Kmeans en un rango de cluster (2, 10)
# Conservar el resultado de "inertial" para utilizar como métrica de seleccion
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

def find_best_model(X, max_clusters=10):

    n_clusters = list(range(2, max_clusters+1)) ## --> [2, 3, 4, 5, 6, 7, 8, 9, 10]
    ensayos = len(n_clusters)                   ## --> 9 (Cantidad de elementos en la lista de clusters)

    # Arma un array con 9 elementos todos ceros, 
    # que se sobreescribirán por un lado con los errore 
    # y por el otro con el score
    array_error = np.zeros((ensayos)) ## --> array([0., 0., 0., 0., 0., 0., 0., 0., 0.])
    array_score = np.zeros((ensayos)) ## --> array([0., 0., 0., 0., 0., 0., 0., 0., 0.])

    for i in range(ensayos): ## --> ensayos = 9
        # En cada iteración se entrenará un modelo de Kmeans,
        # varieando la cantidad de cluster a medida que itere
        # 'k-means++': selecciona los centroides de clúster iniciales.
        #  random_state=0, determina la generación de números aleatorios para la inicialización del centroide, al ser cero
        # producirá los mismos resultados en diferentes llamadas.
        # Entrena con fit
        # Calcula el error con kmeans.inertia_, y a su vez se guarda el array_error.
        # Calcula el score con silhouette_score(X, kmeans.labels_), y a su vez se guarda el array_score
        kmeans = KMeans(n_clusters=n_clusters[i], init="k-means++", random_state=0)
        kmeans.fit(X)
        array_error[i] = kmeans.inertia_
        array_score[i] = silhouette_score(X, kmeans.labels_)

    return array_error, array_score, n_clusters

In [ ]:
# Invocamos la función find_best_model, pasándole los valores de X
array_error, array_score, n_clusters = find_best_model(X_kmeans)

In [ ]:
# Dibujar la evolucion del error (inertia) y deteminar el 
# punto de quiebre (elbow point) para deteminar el mejor k
def graficar_punto_codo(array_error, array_score, n_clusters):
  # Crea el espacio para dibujar y del gráfico.
  fig = plt.figure()
  ax = fig.add_subplot()

  # Se gráfica dos líneas en un mismo gráfico.
  # n_clusters = [2, 3, 4, 5, 6, 7, 8, 9, 10]
  # array_error/array_error.max(), divide cada error por el máximo error
  ax.plot(n_clusters, array_error/array_error.max(), label='error') 
  ax.plot(n_clusters, array_score, label='score')
  # Grilla de fondo y color
  ax.grid('dashed')
  # Leyenda
  ax.legend()
  # Muestra el gráfico
  plt.show()

  # Mejor cluster, se obtiene por la ubicación del máximo score.
  # Para ello, en el array que contiene todos los clusters probados n_clusters
  # Accedemos con cortechetes y se le indica el máximo score con argmax() --> array_score.argmax()
  best_cluster = n_clusters[array_score.argmax()]
  print('El mejor resultado se consigue con clusters=', best_cluster)

In [ ]:
graficar_punto_codo(array_error, array_score, n_clusters)

In [ ]:
kmeans = KMeans(n_clusters=4, init="k-means++", random_state=0)
kmeans.fit(X_kmeans)
labels = kmeans.predict(X_kmeans)

In [ ]:
# Copia
df3 = df2.copy()

In [ ]:
# Agregar una nueva columna con el nombre de segmentación, columna que agrupa a cada cliente por 
# la categoría encontrada en el modelo.
df3['segmentacion'] = labels
df3.head()

In [ ]:
# Calcular la exactitud (accuracy) del modelo
from sklearn.metrics import silhouette_score
silhouette_score (X_kmeans, kmeans.labels_)

### Evaluar modelo

In [ ]:
# Observar la relación entre el ingreso (income) y el débito
# pero ahora coloreando con "hue" los puntos según su clase de custseg
sns.scatterplot(data=df3, x='CONSUMO_COMBUSTIBLE_COMB', y='EMISIONES_CO2', hue='segmentacion', palette='bright')
plt.show()

In [ ]:
# Validar modelo
# Array de prueba
X_prueba = pd.DataFrame({'TAMAÑO_MOTOR': [3.0],	'CILINDROS':[6], 'CONSUMO_COMBUSTIBLE_CIUDAD':[11.5],
                         'CONSUMO_COMBUSTIBLE_HWY':[7.9],	'CONSUMO_COMBUSTIBLE_COMB':[11.9], 
                         'CONSUMO_COMBUSTIBLE_COMB_MPG':[23], 'EMISIONES_CO2':[200]})
						
# Se utliza el modelo con  kmeans.predict()
# Pasandoles los valores 
labels = kmeans.predict(X_prueba.values)
labels

### Conclusión: En este ejemplo, se puede apreciar que el modelo creado con Kmeans agrupa los clusters en forma circular, no estaría separando los datos por cada recta.

### **Crear un modelo con GaussianMixture**
Objetivo: Separar las rectas en 3 conjuntos (clusters) distintos utilizando algún algoritmo de segmentación GaussianMixture.

In [ ]:
# Se hace una copia
df4 = df2.copy()

In [ ]:
#Librerias a implementar
from sklearn.mixture import GaussianMixture

# Se selecciona las columnas relacionadas como: 'CONSUMO_COMBUSTIBLE_COMB', 'EMISIONES_CO2'
# De esas columnas sólo se necesita los valores, por eso se usa .values
X_gaussian_mixture = df4[['CONSUMO_COMBUSTIBLE_COMB', 'EMISIONES_CO2']].values

# Se crea el objeto gmm a partir de la clase GaussianMixture
# Donde se especifica los siguientes parámetros:
# n_components=3, cantidad de grupos a formar, son 3, para separar los datos por la 3 rectas del gráfico.
# covariance_type, cadena que describe el tipo de parámetros de covarianza a utilizar
# Es 'full', porque cada componente tiene su propia matriz de covarianza general.
gmm = GaussianMixture(n_components=3, covariance_type='full')

# Se entrena con fit, los valores de X_gaussian_mixture
gmm.fit(X_gaussian_mixture)

# Los resultados de la agrupación se guardan en el DataFrame df2, en la columna 'grupo_consumo'
df4["grupo_consumo"] = gmm.predict(X_gaussian_mixture)
df4.head()

### Evaluar modelo

In [ ]:
# Observar la relación entre el 'CONSUMO_COMBUSTIBLE_COMB' y 'EMISIONES_CO2'
# pero ahora coloreando con "hue" los puntos según su clase de "grupo_consumo"
sns.scatterplot(data=df4, x='CONSUMO_COMBUSTIBLE_COMB', y='EMISIONES_CO2', hue='grupo_consumo', palette='bright')
plt.show()

### Conclusión: En este ejemplo, se puede apreciar que el modelo creado con GaussianMixture agrupa los datos por cada recta que le corresponde.